## Yahoo Fundamental

## STOCKS



In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import talib
import math
from datetime import datetime, timedelta
# from scipy.signal import argrelextrema
# import plotly.graph_objects as go
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
from bs4 import BeautifulSoup
import requests
from django.core.validators import RegexValidator
from django.http import JsonResponse, Http404

# S&P 500 → ^GSPC
# Dow Jones → ^DJI
# Nasdaq 100 → ^NDX
# Russell 2000 → ^RUT
# DAX (Alemanha) → ^GDAXI
# Definir o ticker da ação
symbol = "AAPL"  # Apple Inc.
# symbol = "^GSPC"  # Apple Inc.
# symbol = "XLK"  # Apple Inc.

# Definir período e intervalo
period = "1y"       # Período de 1 mês
interval = "1d"      # Dados diários


In [7]:
yahoo_data_earnings_history = yf.Ticker(symbol).earnings_history
yahoo_data_earnings_estimate = yf.Ticker(symbol).earnings_estimate
yahoo_data_earnings_dates = yf.Ticker(symbol).earnings_dates

In [8]:
yahoo_data_earnings_dates

,EPS Estimate,Reported EPS,Surprise(%),Event Type
Earnings Date,,,,
2025-05-01 16:30:00-04:00,1.63,1.65,1.41,Earnings
2025-02-27 12:00:00-05:00,NaN,NaN,NaN,Meeting
2025-01-30 16:31:00-05:00,2.35,2.40,2.15,Earnings
2024-10-31 16:31:00-04:00,1.60,1.64,2.35,Earnings
2024-08-01 16:30:00-04:00,1.35,1.40,3.99,Earnings
2024-05-02 16:31:00-04:00,1.50,1.53,1.97,Earnings
2024-02-01 16:00:00-05:00,2.10,2.18,3.90,Earnings
2023-11-02 16:30:00-04:00,1.39,1.46,4.92,Earnings
2023-08-03 16:30:00-04:00,1.19,1.26,5.49,Earnings


In [9]:
yahoo_data_earnings_dates.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 12 entries, 2025-05-01 16:30:00-04:00 to 2022-10-27 16:30:00-04:00
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   EPS Estimate  11 non-null     float64
 1   Reported EPS  11 non-null     float64
 2   Surprise(%)   11 non-null     float64
 3   Event Type    12 non-null     object 
dtypes: float64(3), object(1)
memory usage: 480.0+ bytes


In [15]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# --- assume que seu DataFrame já se chama df ---
df_plot = yahoo_data_earnings_dates.sort_index()
df_plot = df_plot[df_plot["Event Type"] == "Earnings"].copy()

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=df_plot.index, y=df_plot["EPS Estimate"], mode="lines+markers", name="EPS Estimate"),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(x=df_plot.index, y=df_plot["Reported EPS"], mode="lines+markers", name="Reported EPS"),
    secondary_y=False,
)
fig.add_trace(
    go.Bar(x=df_plot.index, y=df_plot["Surprise(%)"], name="Surpresa (%)", opacity=0.4),
    secondary_y=True,
)

fig.update_layout(
    title="Evolução de Earnings: Estimate vs Reported vs Surpresa",
    xaxis_title="Data",
    yaxis_title="EPS",
)
fig.update_yaxes(title_text="Surpresa (%)", secondary_y=True)

fig.show()


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [12]:
yahoo_data_earnings_history

,epsActual,epsEstimate,epsDifference,surprisePercent
quarter,,,,
2024-09-30,0.97,0.94654,0.02,0.0248
2024-12-31,2.40,2.34102,0.06,0.0252
2025-03-31,1.65,1.62253,0.03,0.0169
2025-06-30,1.57,1.42572,0.14,0.1012


In [18]:
!pip install nbformat


In [13]:
yahoo_data_earnings_estimate

,avg,low,high,yearAgoEps,numberOfAnalysts,growth
period,,,,,,
0q,1.75840,1.63,1.82,0.97000,26,0.8128
+1q,2.46401,2.29,2.67,2.40000,22,0.0267
0y,7.38321,7.25,7.47,6.08000,35,0.2143
+1y,7.94575,7.13,9.00,7.38321,38,0.0762


In [23]:
def get_sector_etf_info(sector: str, search_value: str = "info"):
        """
        Return information about symbol ETF sector.
        """
        sector_map = {
            "Technology": "XLK",
            "Financial Services": "XLF",
            "Consumer Cyclical": "XLY",
            "Healthcare": "XLV",
            "Communication Services": "XLC",
            "Industrials": "XLI",
            "Consumer Defensive": "XLP",
            "Energy": "XLE",
            "Real Estate": "XLRE",
            "Basic Materials": "XLB",
            "Utilities": "XLU",
        }

        etf_symbol = sector_map.get(sector)
        if not etf_symbol:
            return "NaN"

        ticker = yf.Ticker(etf_symbol)

        if search_value == "info":
            return ticker.info
        else:
            return ticker.info.get(search_value, "NaN")

In [24]:
# sector = get_sector_etf_info('Utilities')
# sector